In [8]:
import sys
import os
import numpy as np
import pandas as pd
sys.path.append('../..')

from data import constants
from data import cro_dataset


In [13]:
############################## CONFIG ##############################
# Task config
TASK = "multi-label" #@param ["multi-label", "binary"]
CATEGORY_LEVEL = 'cro_sub_type_combined' #@param ["cro", "cro_sub_type_combined"]
MODEL_TYPE = "baseline" #@param ["baseline", "transformer"]
MODEL_NAME = "svm"

# Dataset config
FILTER_OP = True #@param { type: "boolean"}
SCENARIO = "realistic" #@param [ "optimistic", "efficient-realistic", "realistic"]

if SCENARIO == "optimistic":
  TRAIN_NEG_SAMPLING_STRATEGY = "None"
  TEST_NEG_SAMPLING_STRATEGY = "None"

elif SCENARIO == "efficient-realistic":
  TRAIN_NEG_SAMPLING_STRATEGY = "only_OP"
  TEST_NEG_SAMPLING_STRATEGY = "all"

elif SCENARIO == "realistic":
  TRAIN_NEG_SAMPLING_STRATEGY = "all"
  TEST_NEG_SAMPLING_STRATEGY = "all"

# Evaluation metric config. See for context: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html
AVERAGING_STRATEGY = 'macro' #@param ["micro",  "macro", "weighted"]

RESULTS_FILE_NAME = f"{CATEGORY_LEVEL}_{TASK}_results.csv"

# To make the notebook reproducible (not guaranteed for pytorch on different releases/platforms!)
SEED_VALUE = 42

LOCAL_DIR = "/Users/david/Nextcloud/Dokumente/Education/Uni Bern/Master Thesis/Analyzing Financial Climate Disclosures with NLP/Methodology/"
DATA_DIR = os.path.join(LOCAL_DIR, "data", "labels")
train_docs, train_doc_labels, test_docs, test_doc_labels = cro_dataset.prepare_datasets(
    data_dir=DATA_DIR,
    task=TASK, 
    cro_category_level=CATEGORY_LEVEL, 
    should_filter_op=FILTER_OP, 
    train_neg_sampling_strategy=TRAIN_NEG_SAMPLING_STRATEGY, 
    test_neg_sampling_strategy=TEST_NEG_SAMPLING_STRATEGY, 
    seed_value=SEED_VALUE
)

Loaded dataset. Train: 27811, Test: 28209, Dim: 5


In [14]:
two_stage_preds = pd.read_pickle("/Users/david/Nextcloud/Dokumente/Education/Uni Bern/Master Thesis/Analyzing Financial Climate Disclosures with NLP/Methodology/data/labels/test_predictions.pkl")



In [15]:
two_stage_preds

,labels,binary_pred_probs,binary_pred_labels,multilabel_pred_probs,multilabel_pred_labels
0,0,"[0.9999633431434631, 3.663843745016493e-05]",0,"[0.1372823566198349, 0.12800465524196625, 0.37...","[0, 0, 0, 0, 0]"
1,0,"[0.9999604225158691, 3.947925506508909e-05]",0,"[0.28194326162338257, 0.1939391940832138, 0.19...","[0, 0, 0, 0, 0]"
2,0,"[0.9999651908874512, 3.500411185086705e-05]",0,"[0.15715524554252625, 0.06741552799940109, 0.2...","[0, 0, 0, 0, 0]"
3,0,"[0.9999594688415527, 4.0375220123678446e-05]",0,"[0.12715762853622437, 0.1584809422492981, 0.29...","[0, 0, 0, 0, 0]"
4,0,"[0.9999627470970154, 3.7068552046548575e-05]",0,"[0.12583233416080475, 0.14875911176204681, 0.2...","[0, 0, 0, 0, 0]"
...,...,...,...,...,...
28204,0,"[0.9999518394470215, 4.8412483010906726e-05]",0,"[0.20503439009189606, 0.04615365341305733, 0.2...","[0, 0, 0, 0, 0]"
28205,0,"[0.9999575018882751, 4.237275425111875e-05]",0,"[0.08096785098314285, 0.06605876982212067, 0.3...","[0, 0, 0, 0, 0]"
28206,0,"[0.9998440146446228, 0.00015576154692098498]",0,"[0.24945713579654694, 0.05704844743013382, 0.1...","[0, 0, 0, 0, 0]"
28207,0,"[0.999904215335846, 9.596089512342587e-05]",0,"[0.12392881512641907, 0.07752453535795212, 0.2...","[0, 0, 0, 0, 0]"


In [31]:
df_train = train_doc_labels.merge(train_docs, left_index=True, right_index=True)
df_train_pos = df_train[df_train.loc[:, train_doc_labels.columns].any(axis="columns")]
df_train_pos.to_csv(os.path.join(DATA_DIR, "train_positives.csv"))
df_train_pos

,ACUTE,CHRON,POLICY,MARKET,REPUTATION,text
id,,,,,,
fr_lvmh_moet_hennessy-AR_2018_81_0,0,1,0,0,0,"To accompany its initiatives, the Group is als..."
de_sap-AR_2018_218_31,0,0,0,0,1,We believe that lowering SAP’s carbon emission...
es_telefonica-AR_2018_109_13,0,0,0,1,0,Technological\nNeed for early \nwithdrawal of ...
de_daimler-AR_2017_168_2,0,0,0,1,0,Production and technology risks and opportunit...
de_daimler-AR_2007_102_1,1,0,0,0,0,Production and business processes could also b...
...,...,...,...,...,...,...
gb_british_american_tobacco-AR_2010_48_7,0,1,0,0,0,• Scarcity of skilled staff due to \npopulatio...
gb_reckitt_benckiser_grp-AR_2018_45_3,0,0,0,0,1,Emerging risks:\n• Sustainability – We fail to...
gb_glencore_plc-AR_2017_38_6,1,0,0,0,0,"Our metal and coal assets generate \ntailings,..."


In [70]:
df_test = test_doc_labels.merge(test_docs, left_index=True, right_index=True)
two_stage_preds.index = df_test.index
df_test = df_test.merge(two_stage_preds, left_index=True, right_index=True)
# df_test.reset_index(drop=True, inplace=True)
# df2.reset_index(drop=True, inplace=True)
#df_test = pd.concat([df_test, two_stage_preds], axis=1,ignore_index=True)

rel_idx = test_doc_labels.columns.append(pd.Index(["binary_pred_labels"]))
df_test_pos = df_test[df_test.loc[:, rel_idx].any(axis="columns")]
df_test_pos.to_csv(os.path.join(DATA_DIR, "test_positives.csv"))

In [69]:
df_test_pos

,ACUTE,CHRON,POLICY,MARKET,REPUTATION,text,labels,binary_pred_probs,binary_pred_labels,multilabel_pred_probs,multilabel_pred_labels
id,,,,,,,,,,,
be_anheuser_busch_inbev-AR_2019_47_5,1,0,0,0,0,The 2019 crop year proved to be challenging \n...,1,"[0.06597883999347687, 0.9340211153030396]",1,"[0.6400591731071472, 0.274830162525177, 0.0284...","[1, 0, 0, 0, 0]"
be_anheuser_busch_inbev-AR_2019_48_3,0,1,0,0,0,\nWe believe water and climate \nchange are cl...,1,"[0.9325806498527527, 0.06741942465305328]",0,"[0.13573451340198517, 0.3310471773147583, 0.16...","[0, 0, 0, 0, 0]"
be_anheuser_busch_inbev-AR_2019_74_3,0,0,1,0,0,Climate change or other environmental concerns...,1,"[0.0005886996514163911, 0.9994112849235535]",1,"[0.02149936929345131, 0.027850018814206123, 0....","[0, 0, 1, 0, 0]"
be_anheuser_busch_inbev-AR_2019_74_4,0,0,0,0,0,AB InBev's operations are subject to environme...,0,"[0.0011429983424022794, 0.9988569617271423]",1,"[0.029778996482491493, 0.030067358165979385, 0...","[0, 0, 1, 0, 0]"
be_anheuser_busch_inbev-AR_2019_75_7,1,0,0,0,0,AB InBev’s business and operating results coul...,1,"[0.0005898185190744698, 0.9994100332260132]",1,"[0.8565693497657776, 0.08273645490407944, 0.01...","[1, 0, 0, 0, 0]"
...,...,...,...,...,...,...,...,...,...,...,...
gb_vodafone_grp-AR_2019_51_7,1,0,0,0,0,Emerging threats:\nWe could be impacted by an ...,1,"[0.0006779214018024504, 0.9993219971656799]",1,"[0.604487419128418, 0.30425846576690674, 0.024...","[1, 0, 0, 0, 0]"
gb_vodafone_grp-AR_2019_53_13,0,0,0,0,0,Achieving the required reductions in GHG emiss...,0,"[0.0035695356782525778, 0.9964303374290466]",1,"[0.02340106852352619, 0.0362522229552269, 0.61...","[0, 0, 1, 0, 0]"
gb_vodafone_grp-AR_2019_53_15,1,1,1,0,0,Climate change poses a number of potential ris...,1,"[0.0005755277234129608, 0.9994243383407593]",1,"[0.5218310356140137, 0.3767685294151306, 0.032...","[1, 1, 0, 0, 0]"
